In [1]:
from tensorflow.contrib.keras.python.keras.models import Sequential
from tensorflow.contrib.keras.python.keras.layers.recurrent import SimpleRNN, LSTM, GRU
from tensorflow.contrib.keras.python.keras.layers.core import Dense, Dropout, Flatten

from tensorflow.contrib.keras.python.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.contrib.keras.python.keras.optimizers import RMSprop, Adadelta, Adam


from tensorflow.contrib.keras.python.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.contrib.keras.python.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.contrib.keras.python.keras.models import load_model
from tensorflow.contrib.keras.python.keras.preprocessing import image
import numpy as np
import os
import glob

from IPython.display import clear_output

# Confusion matrix
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt

import time
import pickle

In [2]:
# network
TRAIN = 1
n_classes = 7
# n_classes = 6
classes=np.array(('angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise'))
# classes=np.array(('angry', 'disgust', 'fear', 'neutral', 'sad', 'surprise'))

# data
REBUILD_DATA = 0
size = 224
video_length = 48
features_length = 2048

# training
PATH_TRAIN = 'E:/EmotiW2017/lstm/aligned/Train_AFEW_original'
# PATH_TRAIN = 'E:/EmotiW2017/lstm/aligned/Train_AFEW_original'
# PATH_TRAIN = 'E:/EmotiW2017/lstm/aligned/Train_AFEW_N'
NPY_X_TRAIN_DATA = 'training_data_x_resnet50_aligned.npy'
NPY_Y_TRAIN_DATA = 'training_data_y_resnet50_aligned.npy'
# NPY_X_TRAIN_DATA = 'training_data_x_resnet50_aligned_aug.npy'
# NPY_Y_TRAIN_DATA = 'training_data_y_resnet50_aligned_aug.npy'
# NPY_X_TRAIN_DATA = 'training_data_x_inceptionv3_aligned.npy'
# NPY_Y_TRAIN_DATA = 'training_data_y_inceptionv3_aligned.npy'

# validation
# PATH_VAL = 'E:/EmotiW2017/lstm/aligned/Val_AFEW_original'
PATH_VAL = 'E:/EmotiW2017/lstm/aligned/Val_AFEW_original'
NPY_X_VAL_DATA = 'val_data_x_resnet50_aligned.npy'
NPY_Y_VAL_DATA = 'val_data_y_resnet50_aligned.npy'
# NPY_X_VAL_DATA = 'val_data_x_inceptionv3_aligned.npy'
# NPY_Y_VAL_DATA = 'val_data_y_inceptionv3_aligned.npy'

# model
WEIGHT_FILE = 'resnet50_lstm_aligned_3.hdf5'
MODEL_FILE = 'resnet50_lstm_aligned_3.h5'
TRAINING_HIST = 'resnet50_lstm_aligned_3.pickle'
# WEIGHT_FILE = 'inceptionv3_lstm_aligned.hdf5'
# MODEL_FILE = 'inceptionv3_lstm_aligned.h5'
# TRAINING_HIST = 'inceptionv3_lstm_aligned.pickle'

# Feature Extraction with ResNet50

In [3]:
model = load_model('ResNet50_ImageNet.hdf5')
# model = load_model('InceptionV3_ImageNet.hdf5')
# model = InceptionV3(weights='imagenet', include_top=False)
# model.save('InceptionV3_ImageNet.hdf5')

C:\Program Files\Anaconda3\lib\site-packages\tensorflow\contrib\keras\python\keras\models.py:269: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [4]:
def extract_feature(img_path, model, size):
    img = image.load_img(img_path, target_size=(size, size))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)

    features = model.predict(x)
    
    return features[0][0][0]

# features = extract_feature('duong.jpg', model, size)
# features.shape

# Emotion to Integer Convertor

In [5]:
def emo2int(emo):
    if n_classes == 7:
        if emo == 'Angry':
            return 0
        elif emo == 'Disgust':
            return 1
        elif emo == 'Fear':
            return 2
        elif emo == 'Happy':
            return 3
        elif emo == 'Neutral':
            return 4
        elif emo == 'Sad':
            return 5
        else:
            return 6
    elif n_classes == 6:
        if emo == 'Angry':
            return 0
        elif emo == 'Disgust':
            return 1
        elif emo == 'Fear':
            return 2
        elif emo == 'Neutral':
            return 3
        elif emo == 'Sad':
            return 4
        else:
            return 5
    else:
        if emo == 'angry_happy_neutral':
            return 0
        else:
            return 1

# Confusion matrix

In [6]:
def plot_confusion_matrix(y_test, y_pred, classes,
                          normalize=False,
                          title='Unnormalized confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    cm = confusion_matrix(y_test, y_pred)
    np.set_printoptions(precision=2)
    
    plt.figure()
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = np.round(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], 2)

    thresh = cm.min() + (cm.max() - cm.min()) / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()

# Training Data Feature Extraction

In [7]:
# if REBUILD_DATA == 0 and os.path.isfile(NPY_X_TRAIN_DATA) and os.path.isfile(NPY_Y_TRAIN_DATA):
#     print('>>> loading training data...', end='')
#     X_train = np.load(NPY_X_TRAIN_DATA)
#     y_train = np.load(NPY_Y_TRAIN_DATA)
#     print('done.')
# else:
#     # Count # of videos
#     n_vids_train = 0

#     for emo in os.listdir(PATH_TRAIN):
#         full_emo = PATH_TRAIN + '/' + emo
#         if os.path.isdir(full_emo):
#             for vid in os.listdir(full_emo):
#                 full_vid = full_emo + '/' + vid

#                 n_vids_train += 1
    
#     # Read images and extract features
#     vid_indx = 0
    
#     X_train = np.zeros((n_vids_train, video_length, features_length))
#     y_train = []

#     for emo in os.listdir(PATH_TRAIN):
#         full_emo = PATH_TRAIN + '/' + emo
#         if os.path.isdir(full_emo):
#             for vid in os.listdir(full_emo):
#                 full_vid = full_emo + '/' + vid

#                 # Extract features
#                 fra_indx = 0
#                 for fra in os.listdir(full_vid):
#                     if fra_indx < video_length:
#                         full_fra = full_vid + '/' + fra
#                         if os.path.isfile(full_fra):
#                             # Clear screen
#                             clear_output(wait=True)
#                             print('> extracting features for video #%d/%d at frame #%d/%d' % (vid_indx + 1, n_vids_train, fra_indx + 1, video_length))
                            
#                             features = extract_feature(full_fra, model, size)
                            
#                             # Add extracted features to dataset
#                             X_train[vid_indx, fra_indx, :] = features

#                             fra_indx += 1
#                     else:
#                         break

#                 # Get the label for current video
#                 y_train.append(emo2int(emo))

#                 vid_indx += 1

#     y_train = np.array(y_train)
    
#     clear_output(wait=True)
#     print('>>> processed %d videos' % (n_vids_train))

#     np.save(NPY_X_TRAIN_DATA, X_train)
#     np.save(NPY_Y_TRAIN_DATA, y_train)

print('>>> loading training data...', end='')
X_train = np.load(NPY_X_TRAIN_DATA)
y_train = np.load(NPY_Y_TRAIN_DATA)
print('done.')
    
print('\tshapes: {}, {}'.format(X_train.shape, y_train.shape))
print('\ttypes:  {}, {}'.format(X_train.dtype, y_train.dtype))
print('\tmemory: {} MB, {} MB'.format(X_train.nbytes / 1048576, y_train.nbytes / 1048576))

>>> loading training data...done.
	shapes: (726, 48, 2048), (726,)
	types:  float64, int32
	memory: 544.5 MB, 0.00276947021484375 MB


# Validation Data Feature Extraction

In [8]:
if REBUILD_DATA == 0 and os.path.isfile(NPY_X_VAL_DATA) and os.path.isfile(NPY_Y_VAL_DATA):
    print('>>> loading validation data...', end='')
    X_val = np.load(NPY_X_VAL_DATA)
    y_val = np.load(NPY_Y_VAL_DATA)
    print('done.')
else:
    # Count # of videos
    n_vids_val = 0

    for emo in os.listdir(PATH_VAL):
        full_emo = PATH_VAL + '/' + emo
        if os.path.isdir(full_emo):
            for vid in os.listdir(full_emo):
                full_vid = full_emo + '/' + vid

                n_vids_val += 1
    
    # Read images and extract features
    vid_indx = 0
    
    X_val = np.zeros((n_vids_val, video_length, features_length))
    y_val = []

    for emo in os.listdir(PATH_VAL):
        full_emo = PATH_VAL + '/' + emo
        if os.path.isdir(full_emo):
            for vid in os.listdir(full_emo):
                full_vid = full_emo + '/' + vid

                # Extract features
                fra_indx = 0
                for fra in os.listdir(full_vid):
                    if fra_indx < video_length:
                        full_fra = full_vid + '/' + fra
                        if os.path.isfile(full_fra):
                            # Clear screen
                            clear_output(wait=True)
                            print('> extracting features for video #%d/%d at frame #%d/%d' % (vid_indx + 1, n_vids_val, fra_indx + 1, video_length))
                            
                            features = extract_feature(full_fra, model, size)
                            
                            # Add extracted features to dataset
                            X_val[vid_indx, fra_indx, :] = features

                            fra_indx += 1
                    else:
                        break

                # Get the label for current video
                y_val.append(emo2int(emo))

                vid_indx += 1

    y_val = np.array(y_val)
    
    clear_output(wait=True)
    print('>>> processed %d videos' % (n_vids_val))

    np.save(NPY_X_VAL_DATA, X_val)
    np.save(NPY_Y_VAL_DATA, y_val)
    
print('\tshapes: {}, {}'.format(X_val.shape, y_val.shape))
print('\ttypes:  {}, {}'.format(X_val.dtype, y_val.dtype))
print('\tmemory: {} MB, {} MB'.format(X_val.nbytes / 1048576, y_val.nbytes / 1048576))

>>> loading validation data...done.
	shapes: (383, 48, 2048), (383,)
	types:  float64, int32
	memory: 287.25 MB, 0.001461029052734375 MB


In [9]:
y_val

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5,

# LSTM

In [15]:
lstm = Sequential()
lstm.add(LSTM(128, input_shape=(video_length, features_length),
               dropout=0.5, return_sequences=False))
# lstm.add(LSTM(2048, input_shape=(video_length, features_length),
#                dropout=0.5, return_sequences=False))
# lstm.add(Flatten())
lstm.add(Dense(256, activation='relu'))
lstm.add(Dropout(0.5))
lstm.add(Dense(256, activation='relu'))
lstm.add(Dropout(0.5))
lstm.add(Dense(n_classes, activation='softmax'))

In [16]:
print(lstm.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 128)               1114624   
_________________________________________________________________
dense_7 (Dense)              (None, 256)               33024     
_________________________________________________________________
dropout_5 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 256)               65792     
_________________________________________________________________
dropout_6 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 7)                 1799      
Total params: 1,215,239.0
Trainable params: 1,215,239.0
Non-trainable params: 0.0
____________________________________________________________

In [ ]:
batch_size = 172
n_epochs = 10000

learning_rate = 0.0001
early_stop_after = 50

start_time = time.time()

early_stopping = EarlyStopping(monitor='val_acc', patience=early_stop_after, verbose=1)
checkpointer = ModelCheckpoint(monitor='val_acc', filepath='checkpoints/cnn_images' + '_' + str(size) + '_model.{epoch:05d}-{loss:.4f}-{acc:.4f}-{val_loss:.4f}-{val_acc:.4f}.hdf5', verbose=1, save_best_only=True)

# opt = Adam(lr=learning_rate)
opt = RMSprop(lr=learning_rate, decay=10**-6)
lstm.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
if TRAIN == 1:
    train_history = lstm.fit(X_train, y_train,
                             validation_data=(X_val, y_val),
                             shuffle=True, batch_size=batch_size,
                             epochs=n_epochs,
                             verbose=1,
                             callbacks=[checkpointer, early_stopping])
    
    training_time = time.time() - start_time
    print('> Training time: ' + str(training_time) + ' sec')

Train on 726 samples, validate on 383 samples
Epoch 1/10000
726/726 [==============================] - 2s - loss: 2.0003 - acc: 0.1667 - val_loss: 1.9450 - val_acc: 0.1567
Epoch 2/10000
726/726 [==============================] - 1s - loss: 1.9663 - acc: 0.1873 - val_loss: 1.9373 - val_acc: 0.1436
Epoch 3/10000
726/726 [==============================] - 1s - loss: 1.9754 - acc: 0.1446 - val_loss: 1.9302 - val_acc: 0.1749
Epoch 4/10000
726/726 [==============================] - 1s - loss: 1.9643 - acc: 0.1501 - val_loss: 1.9262 - val_acc: 0.2037
Epoch 5/10000
726/726 [==============================] - 1s - loss: 1.9487 - acc: 0.1777 - val_loss: 1.9235 - val_acc: 0.1958
Epoch 6/10000
726/726 [==============================] - 1s - loss: 1.9460 - acc: 0.1736 - val_loss: 1.9217 - val_acc: 0.1749
Epoch 7/10000
726/726 [==============================] - 1s - loss: 1.9452 - acc: 0.1942 - val_loss: 1.9201 - val_acc: 0.1932
Epoch 8/10000
726/726 [==============================] - 1s - loss: 1.94

726/726 [==============================] - 1s - loss: 1.8879 - acc: 0.2107 - val_loss: 1.9029 - val_acc: 0.1958
Epoch 33/10000
726/726 [==============================] - 1s - loss: 1.8837 - acc: 0.2287 - val_loss: 1.9013 - val_acc: 0.2037
Epoch 34/10000
726/726 [==============================] - 1s - loss: 1.9054 - acc: 0.1928 - val_loss: 1.9014 - val_acc: 0.2037
Epoch 35/10000
726/726 [==============================] - 1s - loss: 1.9076 - acc: 0.2287 - val_loss: 1.9020 - val_acc: 0.1984
Epoch 36/10000
726/726 [==============================] - 1s - loss: 1.8957 - acc: 0.2039 - val_loss: 1.9017 - val_acc: 0.2037
Epoch 37/10000
726/726 [==============================] - 1s - loss: 1.8890 - acc: 0.2355 - val_loss: 1.9008 - val_acc: 0.2037
Epoch 38/10000
726/726 [==============================] - 1s - loss: 1.8860 - acc: 0.2369 - val_loss: 1.9001 - val_acc: 0.1932
Epoch 39/10000
726/726 [==============================] - 1s - loss: 1.8825 - acc: 0.2548 - val_loss: 1.9005 - val_acc: 0.1958

# Save model OR Load trained model

In [ ]:
if TRAIN == 1:
    print('> Saving trained model...', end='')
    # Save trained model
    lstm.save_weights(WEIGHT_FILE)
    lstm.save(MODEL_FILE)
    # Save training history
    with open(TRAINING_HIST, 'wb') as f:
        pickle.dump(train_history.history, f)
        
    history_ = train_history.history
    print('done.')
else:
    print('> Loading trained model...', end='')
    # Load trained model
    lstm.load_weights(WEIGHT_FILE)
#     lstm = load_model(MODEL_FILE)
    
    # Load training history
    file = open(TRAINING_HIST, 'rb')
    history_ = pickle.load(file)
    print('done.')

In [ ]:
# Get the latest weight file based on last modified time
lowf = glob.glob('./checkpoints/*.hdf5')
BEST_WEIGHT_FILE = max(lowf, key=os.path.getmtime)

In [ ]:
print('> loading trained model...', end='')
# Load trained model
# lstm = load_model(BEST_WEIGHT_FILE)
lstm.load_weights(BEST_WEIGHT_FILE)

# Load training history
file = open(TRAINING_HIST, 'rb')
history_ = pickle.load(file)
print('done.')

# Validate model again!

In [ ]:
score = lstm.evaluate(X_train, y_train)
print('Train score:', score[0])
print('Train accuracy:', score[1])

score = lstm.evaluate(X_val, y_val)
print('Val score:', score[0])
print('Val accuracy:', score[1])

# Confusion matrix

In [ ]:
y_pred_training_ = lstm.predict(x=X_train, batch_size=batch_size, verbose=1)
y_pred_ = lstm.predict(x=X_val, batch_size=batch_size, verbose=1)

# Saving scores

### Validation

In [ ]:
# Count # of videos
n_vids_val = 0

PATH_VAL_CSV = 'E:/EmotiW2017/lstm/aligned/Val_AFEW_original_scores'

for emo in os.listdir(PATH_VAL):
    full_emo = PATH_VAL + '/' + emo
    if os.path.isdir(full_emo):
        for vid in os.listdir(full_emo):
            n_vids_val += 1

# Read images and extract features
vid_indx = 0

val_names = []

for emo in os.listdir(PATH_VAL):
    full_emo = PATH_VAL + '/' + emo
    if os.path.isdir(full_emo):
        for vid in os.listdir(full_emo):
            full_vid = PATH_VAL_CSV + '/' + emo + '/' + vid
            
            full_vid = full_vid[:-14] + 'csv'

            # Get the name for current video
            val_names.append(full_vid)

            vid_indx += 1

# val_names

In [ ]:
name_indx = 0
cor_pred = 0

for val_f_name in val_names:
    val_scores = y_pred_[name_indx, :]
    cor_pred += (np.argmax(y_pred_[name_indx, :]) == y_val[name_indx])
    
    np.savetxt(val_f_name, val_scores)
    
    name_indx += 1
    
cor_pred/len(y_val)

### Training

In [ ]:
# Count # of videos
n_vids_train = 0

PATH_TRAIN_CSV = 'E:/EmotiW2017/lstm/aligned/Train_AFEW_original_scores'

for emo in os.listdir(PATH_TRAIN):
    full_emo = PATH_TRAIN + '/' + emo
    if os.path.isdir(full_emo):
        for vid in os.listdir(full_emo):
            n_vids_train += 1

# Read images and extract features
vid_indx = 0

train_names = []

for emo in os.listdir(PATH_TRAIN):
    full_emo = PATH_TRAIN + '/' + emo
    if os.path.isdir(full_emo):
        for vid in os.listdir(full_emo):
            full_vid = PATH_TRAIN_CSV + '/' + emo + '/' + vid
            
            full_vid = full_vid[:-14] + 'csv'

            # Get the name for current video
            train_names.append(full_vid)

            vid_indx += 1

# train_names

In [ ]:
name_indx = 0
cor_pred = 0

for train_f_name in train_names:
    train_scores = y_pred_training_[name_indx, :]
    cor_pred += (np.argmax(y_pred_training_[name_indx, :]) == y_train[name_indx])
    
    np.savetxt(train_f_name, train_scores)
    
    name_indx += 1
        
cor_pred/len(y_train)

In [ ]:
y_pred_training = np.argmax(y_pred_training_, axis=1)
y_pred = np.argmax(y_pred_, axis=1)

plot_confusion_matrix(y_test=y_train, y_pred=y_pred_training,
                      classes=classes,
                      normalize=True,
                      title='Normalized confusion matrix - training dataset',
                      cmap=plt.cm.Reds)

plot_confusion_matrix(y_test=y_val, y_pred=y_pred,
                      classes=classes,
                      normalize=True,
                      title='Normalized confusion matrix - validation dataset')

plot_confusion_matrix(y_test=y_val, y_pred=y_pred,
                      classes=classes,
                      normalize=False,
                      title='Unnormalized confusion matrix - validation dataset')

np.sum(y_pred == y_val)/len(y_val)

# Training history

In [ ]:
# list all data in history
print(history_.keys())
# summarize history for accuracy
plt.plot(history_['acc'])
plt.plot(history_['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history_['loss'])
plt.plot(history_['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()